# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/cities_weather_data.csv")
weather_data

,City,Cloud Coverage,City Country,City DateTime,Humidity,City Latitude,City Longitude,Max Temperature,Wind Speed
0,Faanui,100,PF,1637639093,74,-16.4833,-151.7500,81.34,8.84
1,Tuktoyaktuk,40,CA,1637639093,77,69.4541,-133.0374,-5.80,10.36
2,Hilo,90,US,1637639094,96,19.7297,-155.0900,73.13,4.61
3,Saint-Joseph,17,RE,1637639094,67,-21.3667,55.6167,77.36,4.83
4,Wagar,18,SD,1637639095,75,16.1525,36.2032,71.24,10.36
...,...,...,...,...,...,...,...,...,...
558,Kaeo,95,NZ,1637639329,65,-35.1000,173.7833,68.45,4.43
559,Camaná,100,PE,1637639329,86,-16.6228,-72.7111,65.61,5.28
560,Devils Lake,1,US,1637639330,79,48.1128,-98.8651,29.95,7.00
561,Coruripe,13,BR,1637639330,80,-10.1256,-36.1756,76.75,8.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use Lat and Lng as locations and Humidity as the weight
locations = weather_data[["City Latitude", "City Longitude"]]
weights = weather_data["Humidity"]

In [5]:
# Add heatmap layer to map
# Customize the size of the figure
figure_layout = {
    'width': "100%",
    'height': '400px',
    'padding': '1px',
}

# fig=gmaps.figure()
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
fig.add_layer(heatmap_layer)

# Set maximum intensity and point_radius settings
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

# Display heatmap
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the cities to find ideal weather conditions:
# 1 A max temperature lower than 80 degrees but higher than 70
# 2 Wind speed less than 10 mph
# 3 Zero cloudiness
# 4 Humidity is less than 50%

cities_df = weather_data.loc[(weather_data["Max Temperature"] > 70) & 
                            (weather_data["Max Temperature"] < 80) & 
                            (weather_data["Wind Speed"] < 10) & 
                            (weather_data["Cloud Coverage"] == 0) &
                            (weather_data["Humidity"] < 50), :].reset_index(drop=True).dropna()
cities_df


,City,Cloud Coverage,City Country,City DateTime,Humidity,City Latitude,City Longitude,Max Temperature,Wind Speed
0,Kota,0,IN,1637639183,32,25.1833,75.8333,74.21,2.24
1,Pāli,0,IN,1637639194,20,25.7667,73.3333,75.92,2.51
2,Rājāpur,0,IN,1637639236,24,25.3833,81.1500,73.08,5.12
3,Tessaoua,0,NE,1637639237,22,13.7574,7.9874,74.93,8.14
4,Bilma,0,NE,1637639239,24,18.6853,12.9164,71.15,8.90
5,Agadez,0,NE,1637639246,23,19.7500,10.2500,70.45,9.82
6,Manga,0,BF,1637639252,23,11.6636,-1.0731,74.03,6.31
7,Pokaran,0,IN,1637639257,10,26.9167,71.9167,76.46,4.50
8,Córdoba,0,AR,1637639324,49,-31.4135,-64.1811,74.25,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
# Store into variable named hotel_df
hotel_df = cities_df[["City", "City Country", "City Latitude", "City Longitude"]].copy()

# Add a "Hotel Name" column to the DataFrame with null values ---
hotel_df["Hotel Name"] = ""
hotel_df

,City,City Country,City Latitude,City Longitude,Hotel Name
0,Kota,IN,25.1833,75.8333,
1,Pāli,IN,25.7667,73.3333,
2,Rājāpur,IN,25.3833,81.1500,
3,Tessaoua,NE,13.7574,7.9874,
4,Bilma,NE,18.6853,12.9164,
5,Agadez,NE,19.7500,10.2500,
6,Manga,BF,11.6636,-1.0731,
7,Pokaran,IN,26.9167,71.9167,
8,Córdoba,AR,-31.4135,-64.1811,


In [23]:
# Find the closest hotels; define base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters.
params = {"location" : f"{lat},{lng}"
          "type" : "hotel",
          "radius" : 5000,
          "key" : g_key
}

In [28]:
# Loop over each city in the dataframe and make an API request

for index, row in hotel_df.iterrows():
    # Get city name, lat, lnt from df
    lat = row["City Latitude"]
    lng = row["City Longitude"]
    city_name = row["City"]

    # Request the API and get hotel details for each city 
    response = requests.get(base_url, params).json()
    
    try:
        hotel_name = response['results'][0]['name']
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = 'No hotel found'


In [29]:
hotel_df

,City,City Country,City Latitude,City Longitude,Hotel Name
0,Kota,IN,25.1833,75.8333,No hotel found
1,Pāli,IN,25.7667,73.3333,No hotel found
2,Rājāpur,IN,25.3833,81.1500,No hotel found
3,Tessaoua,NE,13.7574,7.9874,No hotel found
4,Bilma,NE,18.6853,12.9164,No hotel found
5,Agadez,NE,19.7500,10.2500,No hotel found
6,Manga,BF,11.6636,-1.0731,No hotel found
7,Pokaran,IN,26.9167,71.9167,No hotel found
8,Córdoba,AR,-31.4135,-64.1811,No hotel found


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
